# New York AirBnB

__Import statements__

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import (
    train_test_split
)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    FunctionTransformer
)
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer

from sklearn.feature_selection import RFECV

from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge, Lasso

from catboost import CatBoostRegressor
from lightgbm.sklearn import LGBMRegressor
from xgboost import XGBRegressor

from sklearn.model_selection import RandomizedSearchCV

from sklearn.ensemble import StackingRegressor

import eli5

from sklearn.feature_selection import SequentialFeatureSelector

__Importing the dataset__

In [ ]:
data = pd.read_csv("../data/raw/airbnb.csv")

In [ ]:
data.head()

In [ ]:
# Drop license, id, last_review
# data.drop(columns=["id", "license", "last_review"], inplace=True)

In [ ]:
data.info()

__Preprocessing__

In [ ]:
data.dropna(subset=["reviews_per_month"], inplace=True)

In [ ]:
data["last_review_year"] = pd.DatetimeIndex(data["last_review"]).year
data["last_review_month"] = pd.DatetimeIndex(data["last_review"]).month
data["last_review_date"] = pd.DatetimeIndex(data["last_review"]).day

In [ ]:
## Trial but failed

# def find_no_of_listings(data):
#     temp_df = pd.DataFrame(data["host_id"].value_counts()).reset_index()

#     temp_df.rename(columns={
#         "index": "host_id",
#         "host_id": "number_of_listings"
#     }, inplace=True)

#     data = pd.merge(data, temp_df, on="host_id")

#     data["number_of_listings"] = pd.cut(
#         data.number_of_listings,
#         bins=[0, 1, 10, np.inf],
#         labels=["1", "2-10", ">10"]
#     )

#     return data

In [ ]:
# # Converting the neighbourhoods occuring less than 86 times to "Others"

# less_frequent_list = X_train["neighbourhood"].value_counts()[86:].index.tolist()
# X_train.loc[X_train.query(
#     "neighbourhood in @less_frequent_list"
# ).index, "neighbourhood"] = "Others"

__Splitting the data__

In [ ]:
train_val_data, test_data = train_test_split(data, test_size=0.2, random_state=123)
train_data, val_data = train_test_split(train_val_data, test_size=0.25, random_state=123)

In [ ]:
train_data.info()

__Feature Type Determination__

In [ ]:
numeric_features = [
    "latitude",
    "longitude",
    "price",
    "minimum_nights",
    "calculated_host_listings_count",
    "availability_365",
    "last_review_year",
    "last_review_date",
    "id",
    "host_id",
    "number_of_reviews",
    "number_of_reviews_ltm"
]

categorical_features = [
    "neighbourhood_group",
    "room_type",
    "last_review_month"
]

text_features = [
    "name"
]

drop_features = [
    "license",
    "host_name",
    "last_review",
    "neighbourhood"
]

passthrough_features = [

]

target_column = "reviews_per_month"

assert (
    len(numeric_features) +
    len(categorical_features) +
    len(text_features) +
    len(drop_features) +
    len(passthrough_features) +
    len([target_column])
) == len(train_data.columns)

In [ ]:
X_train, y_train = train_data.drop(columns=[target_column]), train_data[target_column]
X_val, y_val = val_data.drop(columns=[target_column]), val_data[target_column]
X_train_val, y_train_val = train_val_data.drop(columns=[target_column]), train_val_data[target_column]

In [ ]:
X_train.info()

__Modelling__

In [ ]:
function_transformer = FunctionTransformer(
    np.reshape, kw_args={"newshape": -1}
)

pipe_text_feats = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    function_transformer,
    CountVectorizer(stop_words="english", max_features=30)
)

column_transformer = make_column_transformer(
    (StandardScaler(), numeric_features),
    (OneHotEncoder(handle_unknown="ignore", sparse=False), categorical_features),
    (pipe_text_feats, text_features),
    ("drop", drop_features),
    ("passthrough", passthrough_features)
)

In [ ]:
column_transformer.fit(X_train)

In [ ]:
new_columns = (
    numeric_features +
    column_transformer.named_transformers_["onehotencoder"].get_feature_names_out().tolist() +
    column_transformer.named_transformers_["pipeline"].named_steps["countvectorizer"].get_feature_names_out().tolist() +
    passthrough_features
)

In [ ]:
pd.DataFrame(column_transformer.transform(X_train), columns=new_columns)

In [ ]:
# Credits to Varada K.

# def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
#     """
#     Returns mean and std of cross validation

#     Parameters
#     ----------
#     model :
#         scikit-learn model
#     X_train : numpy array or pandas DataFrame
#         X in the training data
#     y_train :
#         y in the training data

#     Returns
#     ----------
#         pandas Series with mean scores from cross_validation
#     """

#     scores = cross_validate(model, X_train, y_train, **kwargs)

#     mean_scores = pd.DataFrame(scores).mean()
#     std_scores = pd.DataFrame(scores).std()
#     out_col = []

#     for i in range(len(mean_scores)):
#         out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

#     return pd.Series(data=out_col, index=mean_scores.index)

In [ ]:
def cross_val_scores(model, X_train, y_train, X_val, y_val, return_train_score):

    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    score_dict = {
        "r2_test": model.score(X_val, y_val),
        "mse_test": mean_squared_error(y_val, y_val_pred),
        "mape_test": mean_absolute_percentage_error(y_val, y_val_pred)
    }

    if return_train_score:
        y_train_pred = model.predict(X_train)

        score_dict["r2_train"] = model.score(X_train, y_train)
        score_dict["mse_train"] = mean_squared_error(y_train, y_train_pred)
        score_dict["mape_train"] = mean_absolute_percentage_error(y_train, y_train_pred)

    scores_result = pd.Series(score_dict)

    return model, scores_result

In [ ]:
# Adapted from UBC Lecture Notes

def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / true))

In [ ]:
# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

scoring_metrics = {
    "neg RMSE": "neg_root_mean_squared_error",
    "r2": "r2",
    "mape": mape_scorer,
}

__Base Models__

In [ ]:
results = {}

In [ ]:
models = {
    "Dummy": make_pipeline(
        column_transformer, DummyRegressor()
    ),
    "Ridge": make_pipeline(
        column_transformer, Ridge()
    ),
    "Lasso": make_pipeline(
        column_transformer, Lasso()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor(random_state=123)
    ),
    "XGBoost_reg": make_pipeline(
        column_transformer, XGBRegressor(verbosity=0)
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

for model_name, model in models.items():
    _, results[model_name] = cross_val_scores(
        model,
        X_train,
        y_train,
        X_val,
        y_val,
        return_train_score=True
    )

In [ ]:
pd.DataFrame(results)

__Feature Selection__

In [ ]:
results_cat_boost = {}

n_features_list = [30, 35, 40, 45, 50, 55]

for n_features in n_features_list:
    pipe_forward = make_pipeline(
        column_transformer,
        SequentialFeatureSelector(
            Lasso(),
            direction="forward",
            n_features_to_select=n_features
        ),
        CatBoostRegressor(verbose=0)
    )

    _, results_cat_boost[f"rf_forward_fs_{n_features}"] = cross_val_scores(
        pipe_forward, X_train, y_train, X_val, y_val, return_train_score=True
    )

In [ ]:
pd.DataFrame(results_cat_boost)

In [ ]:
rfecv = RFECV(Ridge(), min_features_to_select=40, n_jobs=-1)

pipe_catboost_rfecv = make_pipeline(
    column_transformer, rfecv, CatBoostRegressor(random_state=42, verbose=False)
)

_, results["catboost_rfecv"] = cross_val_scores(
    pipe_catboost_rfecv, X_train, y_train, X_val, y_val, return_train_score=True
)

In [ ]:
pd.DataFrame(results)

In [ ]:
pipe_catboost_rfecv.named_steps["rfecv"].n_features_

__Polynomial Features Regression__

In [ ]:
poly_feats = PolynomialFeatures(degree=2)

pipe_poly = make_pipeline(
    column_transformer, poly_feats, Ridge()
)

_, results["poly_feat"] = cross_val_scores(
    pipe_poly,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True
)

__Hyperparameter Tuning__

_Ridge_

In [ ]:
params = {'ridge__alpha': 10.0 ** np.arange(-2, 6, 1)}

In [ ]:
pipe_ridge = make_pipeline(column_transformer, Ridge())

random_search = RandomizedSearchCV(
    pipe_ridge,
    params,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [ ]:
random_search.fit(X_train_val, y_train_val)

In [ ]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time",
        "mean_score_time",
        "param_ridge__alpha",
        "mean_train_neg RMSE",
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

In [ ]:
pipe_ridge_final = make_pipeline(
    column_transformer,
    Ridge(alpha=1000)
)

pipe_ridge_final.fit(X_train_val, y_train_val)

eli5.explain_weights(pipe_ridge_final.named_steps["ridge"], feature_names=new_columns)

In [ ]:
data = {
    "Importance": pipe_ridge_final.named_steps["ridge"].coef_,
}
pd.DataFrame(data=data, index=new_columns,).sort_values(
    by="Importance", ascending=False
)[:10]

_LGBM_

In [ ]:
params = {
    'lgbmregressor__n_estimators': [10, 100, 1000],
    'lgbmregressor__max_depth': [5, 10, 15]
}

In [ ]:
pipe_lgbm = make_pipeline(column_transformer, LGBMRegressor())

random_search = RandomizedSearchCV(
    pipe_lgbm,
    params,
    n_jobs=-1,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [ ]:
random_search.fit(X_train_val, y_train_val)

In [ ]:
pd.DataFrame(random_search.cv_results_)[[
        "mean_fit_time", 
        "mean_score_time", 
        "param_lgbmregressor__n_estimators", 
        "param_lgbmregressor__max_depth",
        "mean_train_neg RMSE", 
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by='mean_test_r2', ascending=False)

_CatBoost_

In [ ]:
params = {
    "catboostregressor__iterations": [100, 150, 200],
    "catboostregressor__learning_rate": [0.03, 0.1],
    "catboostregressor__depth": [2, 4, 6, 8],
    "catboostregressor__l2_leaf_reg": [0.2, 0.5, 1, 3]
}

In [ ]:
pipe_catboost = make_pipeline(column_transformer, CatBoostRegressor(verbose=False))

random_search = RandomizedSearchCV(
    pipe_catboost,
    params,
    n_jobs=-1,
    n_iter=30,
    return_train_score=True,
    scoring=scoring_metrics,
    refit="r2"
)

In [ ]:
random_search.fit(X_train_val, y_train_val)

In [ ]:
pd.DataFrame(random_search.cv_results_)[
    [
        "mean_fit_time",
        "mean_score_time",
        "param_catboostregressor__iterations",
        "param_catboostregressor__learning_rate",
        "param_catboostregressor__depth",
        "param_catboostregressor__l2_leaf_reg",
        "mean_train_neg RMSE",
        "std_train_neg RMSE",
        "mean_test_mape",
        "mean_train_mape",
        "mean_test_r2",
        "mean_train_r2"
    ]
].sort_values(by="mean_test_r2", ascending=False).T

__Stacking__

In [ ]:
models_selected = {
    "Ridge": make_pipeline(
        column_transformer, PolynomialFeatures(degree=2), Ridge()
    ),
    "Random_Forest_reg": make_pipeline(
        column_transformer, RandomForestRegressor()
    ),
    "lgbm_reg": make_pipeline(
        column_transformer, LGBMRegressor()
    ),
    "catBoost_reg": make_pipeline(
        column_transformer, CatBoostRegressor(verbose=0)
    )
}

In [ ]:
stacking_model = StackingRegressor(list(models_selected.items()))

In [ ]:
stacking_model, results["Stacking"] = cross_val_scores(
    stacking_model,
    X_train,
    y_train,
    X_val,
    y_val,
    return_train_score=True
)

In [ ]:
pd.DataFrame(results)

__Testing on Test Set__